In [53]:
import pandas as pd
import sklearn
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import re
from unicodedata import normalize

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [135]:
teste = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Cópia de Data_Test.csv')
treino = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Cópia de Data_Train.csv')

In [6]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773 entries, 0 to 1772
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       1773 non-null   int64 
 1   Message  1773 non-null   object
 2   SPAM     1773 non-null   bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 29.6+ KB


In [ ]:
treino.groupby(['SPAM']).count()

,ID,Message
SPAM,,
False,841,841
True,932,932


In [126]:
stop_words = [normalize('NFKD', word.encode('latin-1', 'ignore').decode('latin-1')) for word in set(stopwords.words('portuguese'))]

In [139]:
regex = re.compile('[0-9]|\,|\.|\\n|\$|\(|\)|-|\+|\:|\;|\•|\?|\!|\&quot|\&|\#|\https')
teste['Message'] = teste['Message'].apply(lambda x: re.sub(regex, ' ', normalize('NFKD', x.lower().encode('latin-1', 'ignore').decode('latin-1'))).split(' '))

In [140]:
message_list = []
for lista in teste['Message']:
  temp = [palavra for palavra in lista if palavra != '']
  temp = [palavra for palavra in temp if palavra not in stop_words]
  messsage_list.append(temp)
teste['Message'] = message_list